# Neural Machine Translation

**Reference:** Sutskever, Ilya, Oriol Vinyals, and Quoc V. Le. "Sequence to sequence learning with neural networks." In Advances in neural information processing systems, pp. 3104-3112. 2014. ([Paper](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks), [Sample code](https://github.com/tensorflow/nmt))

In [1]:
using Knet, Test, Base.Iterators, IterTools, Random, LinearAlgebra, StatsBase
using AutoGrad: @gcheck  # to check gradients, use with Float64
using Statistics
Knet.atype() = KnetArray{Float32}  # determines what Knet.param() uses.
macro size(z, s); esc(:(@assert (size($z) == $s) string(summary($z),!=,$s))); end # for debugging

@size (macro with 1 method)

## Required utility functions

In [2]:
# Your code here
struct Vocab
    w2i::Dict{String,Int}
    i2w::Vector{String}
    unk::Int
    eos::Int
    tokenizer
end

function Vocab(file::String; tokenizer=split, vocabsize=Inf, mincount=1, unk="<unk>", eos="<s>")
    # Your code here
    counts_dict = Dict{String,Int}()
    w2i_dict = Dict{String,Int}()
    i2w = Vector{String}()

    w2i_function(x) = get!(w2i_dict, x, 1+length(w2i_dict))

    for line in eachline(file)
        for token in tokenizer(line)
            if haskey(counts_dict, token)
                counts_dict[token] += 1
            else
                counts_dict[token] = 1
            end
        end
    end


    if haskey(counts_dict, unk); delete!(counts_dict, unk) end
    if haskey(counts_dict, eos); delete!(counts_dict, eos) end

    for (token, count) in sort!(collect(counts_dict), by=x->x[2], rev=true)[1:Int(min(length(counts_dict), vocabsize-2))]
        if count >= mincount
            w2i_function(token)
            push!(i2w, token)
        end
    end

    w2i_function(unk)
    push!(i2w, unk)    
    w2i_function(eos)
    push!(i2w, eos)    

    Vocab(w2i_dict, i2w, w2i_dict[unk], w2i_dict[eos], tokenizer)
end

Vocab

In [3]:
struct TextReader
    file::String
    vocab::Vocab
end

function Base.iterate(r::TextReader, s=nothing)
    # Your code here
    
    if s == nothing
        s = open(r.file)
    end
    
    if !eof(s)
        return s |> readline |> split |> (x-> [get(r.vocab.w2i, token, r.vocab.unk) for token in x]), s
    else
        close(s)
        return nothing
    end
end

Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

In [4]:
struct Embed; w; end

function Embed(vocabsize::Int, embedsize::Int)
    # Your code here
    Embed(param(embedsize, vocabsize, init=xavier))
end

function (l::Embed)(x)
    # Your code here
    l.w[:, x]
end

struct Linear; w; b; end

function Linear(inputsize::Int, outputsize::Int)
    # Your code here
    Linear(param(outputsize, inputsize, init=xavier), param0(outputsize))
end

function (l::Linear)(x)
    # Your code here
    l.w*x .+ l.b
end

In [5]:
function mask!(a,pad)
    # Your code here
    for r in 1:size(a)[1]
        column_length = size(a)[2]
        for c in 1:column_length
            if a[r, column_length-c+1] != pad
                a[r, column_length-c+3:end] .= 0
                break
            end
        end
    end
    a
end

mask! (generic function with 1 method)

## Part 0. Load data

We will use the Turkish-English pair from the [TED Talks Dataset](https://github.com/neulab/word-embeddings-for-nmt) for our experiments.

In [6]:
datadir = "datasets/tr_to_en"

if !isdir(datadir)
    download("http://www.phontron.com/data/qi18naacl-dataset.tar.gz", "qi18naacl-dataset.tar.gz")
    run(`tar xzf qi18naacl-dataset.tar.gz`)
end

if !isdefined(Main, :tr_vocab)
    tr_vocab = Vocab("$datadir/tr.train", mincount=5)
    en_vocab = Vocab("$datadir/en.train", mincount=5)
    tr_train = TextReader("$datadir/tr.train", tr_vocab)
    en_train = TextReader("$datadir/en.train", en_vocab)
    tr_dev = TextReader("$datadir/tr.dev", tr_vocab)
    en_dev = TextReader("$datadir/en.dev", en_vocab)
    tr_test = TextReader("$datadir/tr.test", tr_vocab)
    en_test = TextReader("$datadir/en.test", en_vocab)
    @info "Testing data"
    @test length(tr_vocab.i2w) == 38126
    @test length(first(tr_test)) == 16
    @test length(collect(tr_test)) == 5029
end

┌ Info: Testing data
└ @ Main In[6]:17


Test Passed
  Expression: length(collect(tr_test)) == 5029
   Evaluated: 5029 == 5029

## Part 1. Minibatching

For minibatching we are going to design a new iterator: `MTData`. This iterator is built
on top of two TextReaders `src` and `tgt` that produce parallel sentences for source and
target languages.

In [7]:
struct MTData
    src::TextReader        # reader for source language data
    tgt::TextReader        # reader for target language data
    batchsize::Int         # desired batch size
    maxlength::Int         # skip if source sentence above maxlength
    batchmajor::Bool       # batch dims (B,T) if batchmajor=false (default) or (T,B) if true.
    bucketwidth::Int       # batch sentences with length within bucketwidth of each other
    buckets::Vector        # sentences collected in separate arrays called buckets for each length range
    batchmaker::Function   # function that turns a bucket into a batch.
end

function MTData(src::TextReader, tgt::TextReader; batchmaker = arraybatch, batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 10, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar source sentence length
    MTData(src, tgt, batchsize, maxlength, batchmajor, bucketwidth, buckets, batchmaker)
end

Base.IteratorSize(::Type{MTData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MTData}) = Base.HasEltype()
Base.eltype(::Type{MTData}) = NTuple{2}

### iterate(::MTData)

Define the `iterate` function for the `MTData` iterator. `iterate` should return a
`(batch, state)` pair or `nothing` if there are no more batches.  The `batch` is a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. The `state` is a pair of `(src_state,tgt_state)` which can be used
to iterate `d.src` and `d.tgt` to get more sentences.  `iterate(d)` without a second
argument should initialize `d` by emptying its buckets and calling `iterate` on the inner
iterators `d.src` and `d.tgt` without a state. Please review the documentation on
iterators from the last project.

To keep similar length sentences together `MTData` uses arrays of similar length sentence
pairs called buckets.  Specifically, the `(src_sentence,tgt_sentence)` pairs coming from
`src` and `tgt` are pushed into `d.buckets[i]` when the length of the source sentence is
in the range `((i-1)*d.bucketwidth+1):(i*d.bucketwidth)`. When one of the buckets reaches
`d.batchsize` `d.batchmaker` is called with the full bucket producing a 2-D batch, the
bucket is emptied and the batch is returned. If `src` and `tgt` are exhausted the
remaining partially full buckets are turned into batches and returned in any order. If the
source sentence length is larger than `length(d.buckets)*d.bucketwidth`, the last bucket
is used.

Sentences above a certain length can be skipped using the `d.maxlength` field, and
transposed `x,y` arrays can be produced using the `d.batchmajor` field.

In [8]:
function Base.iterate(d::MTData, state=nothing)
    if state == nothing
        #Clean buckets
        for i in 1:length(d.buckets)
            d.buckets[i] = []
        end
        #Initialize TextReaders
        state = Vector{Any}([nothing, nothing])
        iterate(d.src)
        iterate(d.tgt)
    end
    
    #Return remaining buckets if one of the sources are exhausted.
    if state isa Number
        for j in state+1:length(d.buckets)
            if !isempty(d.buckets[j])
                return d.batchmaker(d, d.buckets[j]), j
            end
        end
        
        return nothing
    end
        
    while true
        src_sentence, state[1] = iterate(d.src, state[1])
        tgt_sentence, state[2] = iterate(d.tgt, state[2])

        i = min(ceil(Int8, length(src_sentence) / d.bucketwidth), length(d.buckets))
        push!(d.buckets[i], (src_sentence,tgt_sentence))
        
        #If one of the resources is exhausted, start returning batch indices.
        if eof(state[1]) || eof(state[2])
            for j in 1:length(d.buckets)
                if !isempty(d.buckets[j])
                   return d.batchmaker(d, d.buckets[j]), j
                end
            end
            return nothing
        end
        
        if length(d.buckets[i]) == d.batchsize
            batch = d.batchmaker(d, d.buckets[i])
            d.buckets[i] = []
            return batch, state
        end
        
        
        #Skip long source sentences.
        if length(src_sentence) > d.maxlength
            continue
        end      
        
        
    end    
end

### arraybatch

Define `arraybatch(d, bucket)` to be used as the default `d.batchmaker`. `arraybatch`
takes an `MTData` object and an array of sentence pairs `bucket` and returns a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. Note that the sentences in the bucket do not have any `eos` tokens
and they may have different lengths. `arraybatch` should copy the source sentences into
`x` padding shorter ones on the left with `eos` tokens. It should copy the target
sentences into `y` with an `eos` token in the beginning and end of each sentence and
shorter sentences padded on the right with extra `eos` tokens.

In [9]:
function arraybatch(d::MTData, bucket)
    x = []
    y = []

    srclength = 0
    tgtlength = 0 
    for i in 1:length(bucket)
        if length(bucket[i][1]) > srclength; srclength = length(bucket[i][1]) end
        if length(bucket[i][2]) > tgtlength; tgtlength = length(bucket[i][2]) end
    end  
    
    tgtlength += 2 #Because of the additional eos tokens in both of the sides.
    
    for i in 1:length(bucket)
        padded_x = repeat([d.src.vocab.eos], srclength - length(bucket[i][1]))
        append!(padded_x, deepcopy(bucket[i][1]))

        padded_y = [d.tgt.vocab.eos]
        append!(padded_y, deepcopy(bucket[i][2]))
        append!(padded_y, d.tgt.vocab.eos)
        
        append!(padded_y, repeat([d.tgt.vocab.eos], tgtlength - length(padded_y)))       
        
        push!(x, padded_x)
        push!(y, padded_y)
    end
    if d.batchmajor
        return permutedims(hcat(x...))', permutedims(hcat(y...))'
    else
        return permutedims(hcat(x...)), permutedims(hcat(y...))
    end

end

arraybatch (generic function with 1 method)

In [10]:
@info "Testing MTData"
dtrn = MTData(tr_train, en_train)
ddev = MTData(tr_dev, en_dev)
dtst = MTData(tr_test, en_test)

x,y = first(dtst)
@test length(collect(dtst)) == 48
@test size.((x,y)) == ((128,10),(128,24))
@test x[1,1] == tr_vocab.eos
@test x[1,end] != tr_vocab.eos
@test y[1,1] == en_vocab.eos
@test y[1,2] != en_vocab.eos
@test y[1,end] == en_vocab.eos

┌ Info: Testing MTData
└ @ Main In[10]:1


Test Passed
  Expression: y[1, end] == en_vocab.eos
   Evaluated: 18857 == 18857

## Part 2. Sequence to sequence model without attention

In this part we will define a simple sequence to sequence encoder-decoder model for
machine translation.

In [11]:
struct S2S_v1
    srcembed::Embed     # source language embedding
    encoder::RNN        # encoder RNN (can be bidirectional)
    tgtembed::Embed     # target language embedding
    decoder::RNN        # decoder RNN
    projection::Linear  # converts decoder output to vocab scores
    dropout::Real       # dropout probability to prevent overfitting
    srcvocab::Vocab     # source language vocabulary
    tgtvocab::Vocab     # target language vocabulary
end

### S2S_v1 constructor

Define the S2S_v1 constructor using your predefined layer types (Embed, Linear), and the
Knet RNN type. Please review the RNN documentation using `@doc RNN`, paying attention to
the following options in particular: `numLayers`, `bidirectional`, `dropout`, `atype`.
The last one is important if you experiment with array types other than the
default `KnetArray{Float32}`: make sure the RNNs use the same array type as the other
layers. Note that if the encoder is bidirectional, its `numLayers` should be half of the
decoder so that their hidden states match in size.

In [12]:
function S2S_v1(hidden::Int,         # hidden size for both the encoder and decoder RNN
                srcembsz::Int,       # embedding size for source language
                tgtembsz::Int,       # embedding size for target language
                srcvocab::Vocab,     # vocabulary for source language
                tgtvocab::Vocab;     # vocabulary for target language
                layers=1,            # number of layers
                bidirectional=false, # whether encoder RNN is bidirectional
                dropout=0)           # dropout probability
    if bidirectional
          encoder = RNN(srcembsz, hidden; bidirectional=true, numLayers=layers/2, dropout=dropout)
    else
          encoder = RNN(srcembsz, hidden; bidirectional=false, numLayers=layers, dropout=dropout)
    end


    S2S_v1(Embed(length(srcvocab.i2w), srcembsz),
              encoder,
              Embed(length(tgtvocab.i2w), tgtembsz),
              RNN(tgtembsz, hidden; numLayers=layers, dropout=dropout),
              Linear(hidden, size(tgtvocab.i2w)[1]),
              dropout,
              srcvocab,
              tgtvocab)
end

S2S_v1

### S2S_v1 loss function

Define the S2S_v1 loss function that takes `src`, a source language minibatch, and `tgt`,
a target language minibatch and returns either a `(total_loss, num_words)` pair if
`average=false`, or `(total_loss/num_words)` average if `average=true`.

Assume that `src` and `tgt` are integer arrays of size `(B,Tx)` and `(B,Ty)` respectively,
where `B` is the batch size, `Tx` is the length of the longest source sequence, `Ty` is
the length of the longest target sequence. The `src` sequences only contain words, the
`tgt` sequences surround the words with `eos` tokens at the start and end. This allows
columns `tgt[:,1:end-1]` to be used as the decoder input and `tgt[:,2:end]` as the desired
decoder output.

Assume any shorter sentences in the batches have been padded with extra `eos` tokens on
the left for `src` and on the right for `tgt`. Don't worry about masking `src` for the
encoder, it doesn't have a significant effect on the loss. However do mask `tgt` before
`nll`: you do not want the padding tokens to be counted in the loss calculation.

Please review `@doc RNN`: in particular the `r.c` and `r.h` fields can be used to get/set
the cell and hidden arrays of an RNN (note that `0` and `nothing` act as special values).

RNNs take a dropout value at construction and apply dropout to the input of every layer if
it is non-zero. You need to handle dropout for other layers in the loss function or in
layer definitions as necessary.

In [13]:
function (s::S2S_v1)(src, tgt; average=true)
    src_embeddings = s.srcembed(src)
    tgt_embeddings = s.tgtembed(tgt[:,1:end-1])
    
    s.encoder.h = 0
    s.encoder.c = 0

    encoder_hidden_states = s.encoder(src_embeddings)

    s.decoder.c = s.encoder.c
    s.decoder.h = s.encoder.h
            
    decoder_hidden_states = s.decoder(tgt_embeddings)

    prediction = s.projection(reshape(decoder_hidden_states, size(decoder_hidden_states, 1), :))
    
    masked_tgt = tgt[:, 2:end]
    mask!(masked_tgt, s.tgtvocab.eos)

    return nll(reshape(prediction, size(prediction, 1), size(masked_tgt, 1), size(masked_tgt,2)), 
                masked_tgt, 
                average=average)
end

In [17]:
@info "Testing S2S_v1"
Random.seed!(1)
model = S2S_v1(512, 512, 512, tr_vocab, en_vocab; layers=2, bidirectional=true, dropout=0.2)
(x,y) = first(dtst)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.

# My results were (14096.201f0, 1432) and I'm using different vocabulary ordering.
println(model(x,y; average=false))
# @test model(x,y; average=false) == (14096.252f0, 1432) 

(14097.171f0, 1432)


┌ Info: Testing S2S_v1
└ @ Main In[17]:1


### Loss for a whole dataset

Define a `loss(model, data)` which returns a `(Σloss, Nloss)` pair if `average=false` and
a `Σloss/Nloss` average if `average=true` for a whole dataset. Assume that `data` is an
iterator of `(x,y)` pairs such as `MTData` and `model(x,y;average)` is a model like
`S2S_v1` that computes loss on a single `(x,y)` pair.

In [15]:
function loss(model, data; average=true)
    # Your code here
    counts = 0
    total = 0
    for (x, y) in data
        (tot, cnt) = model(x, y, average = false)
        total += tot
        counts += cnt
    end
    if average
        return total/counts
    else
        return total, counts
    end
end

loss (generic function with 1 method)

In [16]:
@info "Testing loss"
println(loss(model, dtst, average=false))
# @test loss(model, dtst, average=false) == (1.0427646f6, 105937)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
# Also, because we do not mask src, different batch sizes may lead to slightly different
# losses. The test above gives (1.0430301f6, 105937) with batchsize==1.

(1.04299156f6, 105937)


┌ Info: Testing loss
└ @ Main In[16]:1


### Training SGD_v1

The following function can be used to train our model. `trn` is the training data, `dev`
is used to determine the best model, `tst...` can be zero or more small test datasets for
loss reporting. It returns the model that does best on `dev`.

In [18]:
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=100) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        println(losses)
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

train! (generic function with 1 method)

You should be able to get under 3.40 dev loss with the following settings in 10
epochs. The training speed on a V100 is about 3 mins/epoch or 40K words/sec, K80 is about
6 times slower. Using settings closer to the Luong paper (per-sentence loss rather than
per-word loss, SGD with lr=1, gclip=1 instead of Adam), you can get to 3.17 dev loss in
about 25 epochs. Using dropout and shuffling batches before each epoch significantly
improve the dev loss. You can play around with hyperparameters but I doubt results will
get much better without attention. To verify your training, here is the dev loss I
observed at the beginning of each epoch in one training session:
`[9.83, 4.60, 3.98, 3.69, 3.52, 3.41, 3.35, 3.32, 3.30, 3.31, 3.33]`

In [ ]:
@info "Training S2S_v1"
epochs = 10
ctrn = collect(dtrn)
trnx10 = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
trn20 = ctrn[1:20]
dev38 = collect(ddev)
# Uncomment this to train the model (This takes about 30 mins on a V100, 60 mins on a T4):
model = train!(model, trnx10, dev38, trn20);
# Uncomment this to save the model:
# Knet.save("s2s_v1.jld2","model",model)
# Uncomment this to load the model:
# model = Knet.load("s2s_v1.jld2","model")

In [35]:
#Knet.save("s2s_v1.jld2","model",model)

In [19]:
model = Knet.load("s2s_v1.jld2","model")

S2S_v1(Embed(P(Knet.KnetArrays.KnetMatrix{Float32}(512,38126))), LSTM(input=512,hidden=512,bidirectional,dropout=0.2), Embed(P(Knet.KnetArrays.KnetMatrix{Float32}(512,18857))), LSTM(input=512,hidden=512,layers=2,dropout=0.2), Linear(P(Knet.KnetArrays.KnetMatrix{Float32}(18857,512)), P(Knet.KnetArrays.KnetVector{Float32}(18857))), 0.2, Vocab(Dict("ağacından" => 35368, "komuta" => 13564, "ellisi" => 25237, "adresini" => 22818, "yüzeyi" => 4049, "paris'te" => 9492, "kafamdaki" => 18788, "yüzeyinde" => 5040, "geçerlidir" => 6610, "kökten" => 7772…), [".", ",", "bir", "ve", "bu", "''", "``", "için", "çok", "da"  …  "karşılaştırılabilir", "ördeğin", "gününüzü", "bağışçı", "istismara", "yaşça", "tedci", "fakültesi'nde", "<unk>", "<s>"], 38125, 38126, split), Vocab(Dict("offend" => 14330, "hyperbole" => 18615, "enjoy" => 1730, "advertisements" => 10004, "chocolate" => 3321, "nicholas" => 9023, "tolstoy" => 17601, "uniformly" => 18616, "favorites" => 5269, "middle-income" => 13396…), [",", ".",

### Generating translations

With a single argument, a `S2S_v1` object should take it as a batch of source sentences
and generate translations for them. After passing `src` through the encoder and copying
its hidden states to the decoder, the decoder is run starting with an initial input of all
`eos` tokens. Highest scoring tokens are appended to the output and used as input for the
subsequent decoder steps.  The decoder should stop generating when all sequences in the
batch have generated `eos` or when `stopfactor * size(src,2)` decoder steps are reached. A
correctly shaped target language batch should be returned.

In [20]:
function (s::S2S_v1)(src::Matrix{Int}; stopfactor = 3)
    # Your code here
    src_embeddings = s.srcembed(src)
    s.encoder.h = 0
    s.encoder.c = 0
      
    encoder_hidden_states = s.encoder(src_embeddings)

    s.decoder.c = s.encoder.c
    s.decoder.h = s.encoder.h
    
    prediction = reshape(hcat(repeat([s.tgtvocab.eos], size(src, 1))...), size(src, 1), 1) #All eos tags.
    prediction_embeddings = s.tgtembed(prediction)
#     prediction_embeddings = reshape(prediction_embeddings, size(prediction_embeddings, 1), size(prediction_embeddings,3), size(prediction_embeddings, 2))
    generated_words = prediction #Start with eos for each batch

    for i in 1:(stopfactor * size(src,2))
        decoder_hidden_states = s.decoder(prediction_embeddings);
        scores = s.projection(reshape(decoder_hidden_states, size(decoder_hidden_states, 1), :))
        prediction = [x[1] for x in argmax(scores, dims=1)]
        generated_words = hcat(generated_words, prediction')
        prediction_embeddings = s.tgtembed(prediction)
        prediction_embeddings = reshape(prediction_embeddings, size(prediction_embeddings,1), size(prediction_embeddings, 3), size(prediction_embeddings, 2))
        
        if (sum(prediction .== s.tgtvocab.eos) == size(prediction, 1))
            break
        end
    end
    generated_words[:, 2:end]    
end

In [21]:
# Utility to convert int arrays to sentence strings
function int2str(y,vocab)
    y = vec(y)
    ysos = findnext(w->!isequal(w,vocab.eos), y, 1)
    ysos == nothing && return ""
    yeos = something(findnext(isequal(vocab.eos), y, ysos), 1+length(y))
    join(vocab.i2w[y[ysos:yeos-1]], " ")
end

int2str (generic function with 1 method)

In [22]:
# Uncomment and run these lines if you get a "CUDNNError: CUDNN_STATUS_INTERNAL_ERROR (code 4)" error from the cell below.
# Knet.save("s2s_v1.jld2","model",model)
# model = Knet.load("s2s_v1.jld2","model")

In [25]:
@info "Generating some translations"
d = MTData(tr_dev, en_dev, batchsize=1) |> collect
(src,tgt) = rand(d)
out = model(src)
println("SRC: ", int2str(src,model.srcvocab))
println("REF: ", int2str(tgt,model.tgtvocab))
println("OUT: ", int2str(out,model.tgtvocab))
# Here is a sample output:
# SRC: çin'e 15 şubat 2006'da ulaştım .
# REF: i made it to china on february 15 , 2006 .
# OUT: i got to china , china , at the last 15 years .

SRC: alice hastalarına çok sıcak , çok anlayışlı <unk> .
REF: alice was very warm , very empathetic with her patients .
OUT: and the adults who are very angry , too many , very much for the <unk> .


┌ Info: Generating some translations
└ @ Main In[25]:1


### Calculating BLEU

BLEU is the most commonly used metric to measure translation quality. The following should
take a model and some data, generate translations and calculate BLEU.

In [26]:
function bleu(s2s,d::MTData)
    d = MTData(d.src,d.tgt,batchsize=1)
    reffile = d.tgt.file
    hypfile,hyp = mktemp()
    for (x,y) in progress(collect(d))
        g = s2s(x)
        for i in 1:size(y,1)
            println(hyp, int2str(g[i,:], d.tgt.vocab))
        end
    end
    close(hyp)
    isfile("multi-bleu.perl") || download("https://github.com/moses-smt/mosesdecoder/raw/master/scripts/generic/multi-bleu.perl", "multi-bleu.perl")
    run(pipeline(`cat $hypfile`,`perl multi-bleu.perl $reffile`))
    return hypfile
end

bleu (generic function with 1 method)

Calculating dev BLEU takes about 100 secs on a T4. We get about 8.0 BLEU which is pretty
low. As can be seen from the sample translations a loss of ~3+ (perplexity ~20+) or a BLEU
of ~8 is not sufficient to generate meaningful translations.

In [27]:
@info "Calculating BLEU"
bleu(model, ddev)

┌ Info: Calculating BLEU
└ @ Main In[27]:1
┣████████████████████┫ [100.00%, 4045/4045, 02:43/02:43, 24.81i/s] 


BLEU = 7.50, 36.5/11.1/4.3/1.8 (BP=1.000, ratio=1.079, hyp_len=89054, ref_len=82502)


It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


"C:\\Users\\atakan\\AppData\\Local\\Temp\\jl_66F2.tmp"

To improve the quality of translations we can use more training data, different training
and model parameters, or preprocess the input/output: e.g. splitting Turkish words to make
suffixes look more like English function words may help. Other architectures,
e.g. attention and transformer, perform significantly better than this simple S2S model.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*